<a href="https://colab.research.google.com/github/thegallier/configs/blob/main/Mistral_7b_instruct_feature_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Note: Responses from local models can be quite slow, especially with 8-bit quantization.

With 4bit quantization, `mistralai/Mistral-7B-Instruct-v0.1` uses about 12GB of VRAM and 8.5GB of RAM. I used a T4-High RAM instance for this notebook.

In [1]:
!pip install git+https://github.com/run-llama/llama_index

  Cloning https://github.com/run-llama/llama_index to /tmp/pip-req-build-kphy25oi
  Running command git clone --filter=blob:none --quiet https://github.com/run-llama/llama_index /tmp/pip-req-build-kphy25oi
  Resolved https://github.com/run-llama/llama_index to commit 6c6f586322b088bcae9005e0a704e9bc4d205055
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━

In [2]:
!pip install transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 17.5 MB/s eta 0:00:00


## Setup

### Data

In [3]:
from llama_index.readers import BeautifulSoupWebReader

url = "https://www.theverge.com/2023/9/29/23895675/ai-bot-social-network-openai-meta-chatbots"

documents = BeautifulSoupWebReader().load_data([url])

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
documents

[Document(id_='bfa99cea-6fad-4f9e-a5a8-3f13baebb545', embedding=None, metadata={'URL': 'https://www.theverge.com/2023/9/29/23895675/ai-bot-social-network-openai-meta-chatbots'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='6ab45f6814c6273daaded8d02387ab77440244732eddbb8d36babf3a9ded88b1', text="The synthetic social network is coming - The VergeSkip to main contentThe VergeThe Verge logo.The Verge homepageThe Verge homepageThe VergeThe Verge logo./Tech/Reviews/Science/Entertainment/MoreMenuExpandThe VergeThe Verge logo.MenuExpandPlatformer/Artificial Intelligence/TechThe synthetic social network is comingThe synthetic social network is coming / Between ChatGPT’s surprisingly human voice and Meta’s AI characters, our feeds may be about to change foreverBy  Casey Newton, a contributing editor who has been writing about tech for over 10 years. He founded Platformer, a newsletter about Big Tech and democracy. Sep 29, 2023, 1:30 PM UTC|CommentsShare

### LLM

This should run on a T4 instance on the free tier

In [32]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.prompts import PromptTemplate
from llama_index.llms import HuggingFaceLLM

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


llm = HuggingFaceLLM(
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    query_wrapper_prompt=PromptTemplate("<s>[INST] {query_str} [/INST] </s>\n"),
    context_window=3900,
    max_new_tokens=256,
    model_kwargs={"quantization_config": quantization_config},
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.2, "top_k": 5, "top_p": 0.95},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
!pip install llama_index

In [2]:
from llama_index import ServiceContext

#service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local")

### Index Setup

In [ ]:
il.com


In [13]:
#!pip install edgartools
from edgar import *


TypeError: ignored

In [5]:
filings=Company("aapl").get_filings(form="10-K")

In [6]:
aapl_html=filings.latest(1).html()

In [7]:
with open("aaplhtml","w") as f:
  f.write(aapl_html)

In [8]:
from llama_index.readers.file.flat_reader import FlatReader
from pathlib import Path

reader = FlatReader()
documents= reader.load_data(Path("aaplhtml"))


In [9]:
from llama_index import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [10]:
from llama_index import SummaryIndex

summary_index = SummaryIndex.from_documents(documents, service_context=service_context)

In [49]:
from llama_index.node_parser import (
    UnstructuredElementNodeParser,
)

node_parser = UnstructuredElementNodeParser(llm=llm)

In [53]:
old
https://medium.com/@jerryjliu98/how-unstructured-and-llamaindex-can-help-bring-the-power-of-llms-to-your-own-data-3657d063e30d

NameError: ignored

In [51]:
raw_nodes_2021 = node_parser.get_nodes_from_documents(documents,llm=llm,embed_model="local")

  0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generati

In [54]:
base_nodes_2021, node_mappings_2021 = node_parser.get_base_nodes_and_mappings(
    raw_nodes_2021
)

In [56]:
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index import VectorStoreIndex

In [57]:
# construct top-level vector index + query engine
vector_index = VectorStoreIndex(base_nodes_2021)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)
vector_query_engine = vector_index.as_query_engine(similarity_top_k=1)

In [60]:
from llama_index.retrievers import RecursiveRetriever

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    node_dict=node_mappings_2021,
    verbose=True,
)
query_engine = RetrieverQueryEngine.from_args(recursive_retriever)

In [61]:
response = query_engine.query("What was the revenue in 2020?")
print(str(response))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Retrieving with query id None: What was the revenue in 2020?
Retrieving text node: (2)

Services net sales include amortization of the deferred value of services bundled in the sales price of certain products.

Total net sales include $8.2 billion of revenue recognized in 2023 that was included in deferred revenue as of September 24, 2022, $7.5 billion of revenue recognized in 2022 that was included in deferred revenue as of September 25, 2021, and $6.7 billion of revenue recognized in 2021 that was included in deferred revenue as of September 26, 2020.

The Company’s proportion of net sales by disaggregated revenue source was generally consistent for each reportable segment in Note 13, “Segment Information and Geographic Data” for 2023, 2022 and 2021, except in Greater China, where iPhone revenue represented a moderately higher proportion of net sales.

Note 3 – Earnings Per Share

The following table shows the computation of basic and diluted earnings per share for 2023, 2022 and 202

In [66]:
response = query_engine.query("How many treasuries did apple hold?")
print(str(response))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Retrieving with query id None: How many treasuries did apple hold?
Retrieving text node: Apple Inc. | 2023 Form 10-K | 31

Apple Inc.

CONSOLIDATED STATEMENTS OF CASH FLOWS

(In millions)

Years ended September 30, 2023 September 24, 2022 September 25, 2021 Cash, cash equivalents and restricted cash, beginning balances $ 24,977   $ 35,929   $ 39,789    Operating activities: Net income 96,995   99,803   94,680   Adjustments to reconcile net income to cash generated by operating activities: Depreciation and amortization 11,519   11,104   11,284   Share-based compensation expense 10,833   9,038   7,906    Other ( 2,227 ) 1,006   ( 4,921 ) Changes in operating assets and liabilities: Accounts receivable, net ( 1,688 ) ( 1,823 ) ( 10,125 ) Vendor non-trade receivables 1,271   ( 7,520 ) ( 3,903 ) Inventories ( 1,618 ) 1,484   ( 2,642 ) Other current and non-current assets ( 5,684 ) ( 6,499 ) ( 8,042 ) Accounts payable ( 1,889 ) 9,448   12,326   Other current and non-current liabilities 3,031

In [67]:
response

Response(response='[/', source_nodes=[NodeWithScore(node=TextNode(id_='919368bc-670d-47d0-afa3-c8d4ce989222', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='639d8250-dc8a-4536-93e0-a0960c9f066f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='dcd4d39a57d46207779b5736fb219986cbebc22c6aa6b397b325cc2477c427b3'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='4bd3d8a7-d7ac-449d-9448-47d83717215d', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='c2989eb96fd5f51ce58d26e0d0d2bc57d8e9f9f76861a076830d2f83a456e328'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='dacffedd-7c96-494b-ae8e-f1e6a25adc03', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='9d3a00881b0683d1cec91e35fa25479ff8ad83e7d3ef14f746d1850ab29c848c')}, hash='50c415621b2336e302fc3c9b03fc9a5e42db72aa3484966f73caf6fe9d86734b', text='Apple Inc. | 2023 Form 10-K | 31\n\nApple Inc.\

In [16]:
from typing import Any

from pydantic import BaseModel
from unstructured.partition.html import partition_html


In [17]:
!pip install InstructorEmbedding

In [18]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=471ff1c16346062c77eb6d92631c4104112cb65f0f0b39adeff3ca4bfdbc8405
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [19]:
from typing import Any, List
from InstructorEmbedding import INSTRUCTOR
from llama_index.embeddings.base import BaseEmbedding


class InstructorEmbeddings(BaseEmbedding):
    def __init__(
        self,
        instructor_model_name: str = "hkunlp/instructor-large",
        instruction: str = "Represent the Computer Science documentation or question:",
        **kwargs: Any,
    ) -> None:
        self._model = INSTRUCTOR(instructor_model_name)
        self._instruction = instruction
        super().__init__(**kwargs)

        def _get_query_embedding(self, query: str) -> List[float]:
            embeddings = self._model.encode([[self._instruction, query]])
            return embeddings[0]

        def _get_text_embedding(self, text: str) -> List[float]:
            embeddings = self._model.encode([[self._instruction, text]])
            return embeddings[0]

        def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
            embeddings = self._model.encode(
                [[self._instruction, text] for text in texts]
            )
            return embeddings

In [20]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.3 MB/s eta 0:00:00


In [33]:
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
from llama_index import ServiceContext

embed_model = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-base-en")

#service_context = ServiceContext.from_defaults(embed_model=embed_model,llm=llm)
service_context = ServiceContext.from_defaults(embed_model="local",llm=llm)

In [66]:
print(llama_index.__version__)

NameError: ignored

In [22]:
embeddings = embed_model.get_text_embedding(
    "It is raining cats and dogs here!"
)


AttributeError: ignored

In [34]:
from llama_index import ServiceContext, set_global_service_context
from llama_index.llms import OpenAI
from llama_index.embeddings import OpenAIEmbedding, HuggingFaceEmbedding
from llama_index.node_parser import (
    SentenceWindowNodeParser,
)
from llama_index.text_splitter import SentenceSplitter

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

# base node parser is a sentence splitter
text_splitter = SentenceSplitter()

#llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-mpnet-base-v2", max_length=512
)
ctx = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    # node_parser=node_parser,
)

In [35]:

set_global_service_context(service_context)

In [45]:
raw_nodes_2021 = node_parser.get_nodes_from_documents(documents,llm=llm,embed_model="local")

In [48]:
len(raw_nodes_2021)

625

In [36]:
import os
import pickle

if not os.path.exists("2021_nodes.pkl"):
    # raw_nodes_2021 = node_parser.get_nodes_from_documents(documents,llm=llm,embed_model="local:BAAI/bge-small-en-v1.5")
    raw_nodes_2021 = node_parser.get_nodes_from_documents(documents,llm=llm,embed_model="local")
    pickle.dump(raw_nodes_2021, open("2021_nodes.pkl", "wb"))
else:
    raw_nodes_2021 = pickle.load(open("2021_nodes.pkl", "rb"))

In [37]:
print(raw_nodes_2021)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Helpful Imports / Logging

In [39]:
from llama_index.response.notebook_utils import display_response

In [30]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Basic Query Engine

### Compact (default)

In [40]:
query_engine = vector_index.as_query_engine(response_mode="compact")

response = query_engine.query("What are apples financial risks wrt to interest rates, inflation and foreign exachange?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** [/

In [40]:
response

Response(response='[/', source_nodes=[NodeWithScore(node=TextNode(id_='2c258a9b-959e-40e2-b250-f004df1b7785', embedding=None, metadata={'filename': 'aaplhtml', 'extension': ''}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='63f9e7b4-523f-4deb-87ba-749fb9c7c4b6', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'filename': 'aaplhtml', 'extension': ''}, hash='822f6a8a96a40174bbc6e9690daac24292415b27d3e1dc614d50b1476417eb69'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='6db01500-8ec8-4233-9af9-8e184e72f53e', node_type=<ObjectType.TEXT: '1'>, metadata={'filename': 'aaplhtml', 'extension': ''}, hash='e40738545909d6218cd2a7a6249a513b602c2d3e5d6795b698a91392f2a4f4a0'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='b2618257-e475-446a-82bf-0960b3d8ddbe', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='b9792722960c95b4e48b01595437dd019d3bee91054e79d370937c2a4cf6441a')}, hash='d0b

### Refine

In [41]:
query_engine = vector_index.as_query_engine(response_mode="refine")

response = query_engine.query("How do OpenAI and Meta differ on AI tools?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** [/

In [42]:
response

Response(response='[/', source_nodes=[NodeWithScore(node=TextNode(id_='74ed0362-7825-411c-8e5a-8f5f99bce915', embedding=None, metadata={'filename': 'aaplhtml', 'extension': ''}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='894f8baf-bba0-41eb-8fab-2270fc0d46c6', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'filename': 'aaplhtml', 'extension': ''}, hash='822f6a8a96a40174bbc6e9690daac24292415b27d3e1dc614d50b1476417eb69'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='5daa3f1f-1e66-4c9d-9f49-71a8173a7110', node_type=<ObjectType.TEXT: '1'>, metadata={'filename': 'aaplhtml', 'extension': ''}, hash='2de86817b1e80a03d579527f1838db04a6560bcaedc17da725bc9df083b9e74d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='f1d19e41-8350-4846-9b26-cdef5e09f687', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='b8767119650fb305c469ce1592af44da3a7cb3fd07d9259094259a50d521da49')}, hash='23f

### Tree Summarize

In [68]:
query_engine2 = vector_index.as_query_engine(response_mode="tree_summarize")

response = query_engine2.query("Does apple have exposure to foreign exhange changes?")

display_response(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** [/

In [69]:
response

Response(response='[/', source_nodes=[NodeWithScore(node=TextNode(id_='bd76a2d1-95db-4351-b287-49f29e065c59', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a384b4c9-9656-4cad-a6e7-b7aa92dccfae', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='458f4f616ef88556bd5bdcfaf788bbfba1aca4762edc538de07853990ef633a0'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='a8751be6-54c3-4280-84b8-71a6e98105ca', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='91d8b35cb6e67f0746b52fb8fbb6db457278d71b152d36a58e8856252eeb45d9'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='c5658617-c624-4dda-82e1-0ef03ec65e1c', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='7fd6ce578bd10222fa4598814e601072116f045a246deb89e8b47042ee560cb8')}, hash='7ddfdf856d367dbb4709dfdf9bb765a02de44cec8af362f4e4c722d4fba49034', text='Additionally, strengthening of foreign currenci

## Router Query Engine

In [43]:
from llama_index.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    )
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document."
    )
)

### Single Selector

In [44]:
from llama_index.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    service_context=service_context,
    select_multi=False
)

response = query_engine.query("What was mentioned about apple and treasury investments?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** [/

In [47]:
response

Response(response='[/', source_nodes=[NodeWithScore(node=TextNode(id_='3887d4fb-5570-426b-81a0-f49dc0c3bc1b', embedding=None, metadata={'filename': 'aaplhtml', 'extension': ''}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='63f9e7b4-523f-4deb-87ba-749fb9c7c4b6', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'filename': 'aaplhtml', 'extension': ''}, hash='822f6a8a96a40174bbc6e9690daac24292415b27d3e1dc614d50b1476417eb69'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='1d9f94c8-7724-44e1-a05b-8bd232bb61f2', node_type=<ObjectType.TEXT: '1'>, metadata={'filename': 'aaplhtml', 'extension': ''}, hash='9f976446933bdc805f8068259f5ad1ffb575aa674dcbe2f0b501bbcd0c7aeb66'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='6db01500-8ec8-4233-9af9-8e184e72f53e', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='e40738545909d6218cd2a7a6249a513b602c2d3e5d6795b698a91392f2a4f4a0')}, hash='d76

In [18]:
response

Response(response='[/', source_nodes=[NodeWithScore(node=TextNode(id_='f21ab7c4-6cc2-4e4e-8028-aa8801088d55', embedding=None, metadata={'URL': 'https://www.theverge.com/2023/9/29/23895675/ai-bot-social-network-openai-meta-chatbots'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='bfa99cea-6fad-4f9e-a5a8-3f13baebb545', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'URL': 'https://www.theverge.com/2023/9/29/23895675/ai-bot-social-network-openai-meta-chatbots'}, hash='6ab45f6814c6273daaded8d02387ab77440244732eddbb8d36babf3a9ded88b1'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='55702518-293c-46fb-9672-96f2955a30fe', node_type=<ObjectType.TEXT: '1'>, metadata={'URL': 'https://www.theverge.com/2023/9/29/23895675/ai-bot-social-network-openai-meta-chatbots'}, hash='062e1c0290495f3617244a5b15ca250fa3971513893a47ed27a146ff47d12bfe')}, hash='e70e318bf83a7619c2a469b7680742bebfb9f3ee6bb0c01480

### Multi Selector

In [48]:
from llama_index.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    service_context=service_context,
    select_multi=True,
)

response = query_engine.query("Summarize apples interest rate, foreign exchange and inflation risk and the hedges they have in 3 bullet points")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

OutOfMemoryError: ignored

## SubQuestion Query Engine

In [ ]:
from llama_index.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    )
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document."
    )
)

In [20]:
import nest_asyncio
nest_asyncio.apply()

In [22]:
from llama_index.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    service_context=service_context,
    verbose=True,
)

response = query_engine.query("What was mentioned about Meta? How Does it differ from how OpenAI is talked about?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ValueError: ignored

## SQL Query Engine

Here, we download and use a sample SQLite database with 11 tables, with various info about music, playlists, and customers. We will limit to a select few tables for this test.

In [23]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [24]:
!curl https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip -O /content/chinook.zip
!unzip /content/chinook.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  298k  100  298k    0     0   301k      0 --:--:-- --:--:-- --:--:--  301k
curl: (3) URL using bad/illegal format or missing URL
Archive:  /content/chinook.zip
  inflating: chinook.db              


In [25]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, select, column

engine = create_engine("sqlite:////content/chinook.db")

In [26]:
from llama_index import SQLDatabase

sql_database = SQLDatabase(engine)

In [27]:
from llama_index.indices.struct_store import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
    service_context=service_context
)

In [28]:
response = query_engine.query("What are some albums? Limit it to 5.")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** [/

In [29]:
response

Response(response='[/', source_nodes=[NodeWithScore(node=TextNode(id_='8ea90dfa-fc61-4ecf-90d6-a3f914dab86a', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='7fe0c6a2457bf8cea47fdc8aee872d27f98f895bb60c80474df51db7f3ac0d64', text="Error: Statement '[/' is invalid SQL.", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'8ea90dfa-fc61-4ecf-90d6-a3f914dab86a': {}, 'sql_query': '[/'})

In [30]:
response = query_engine.query("What are some artists? Limit it to 5.")

display_response(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_to

**`Final Response:`** [/

This last query should be a more complex join

In [31]:
response = query_engine.query("What are some tracks from the artist AC/DC? Limit it to 3")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** [/

In [33]:
response

Response(response='[/', source_nodes=[NodeWithScore(node=TextNode(id_='3effb395-a80a-40ac-953c-4f3b0af64e74', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='7fe0c6a2457bf8cea47fdc8aee872d27f98f895bb60c80474df51db7f3ac0d64', text="Error: Statement '[/' is invalid SQL.", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'3effb395-a80a-40ac-953c-4f3b0af64e74': {}, 'sql_query': '[/'})

In [32]:
print(response.metadata['sql_query'])

[/


## Programs

Depending the LLM, you will have to test with either `OpenAIPydanticProgram` or `LLMTextCompletionProgram`

In [ ]:
from typing import List
from pydantic import BaseModel

from llama_index.program import OpenAIPydanticProgram, LLMTextCompletionProgram

class Song(BaseModel):
    """Data model for a song."""

    title: str
    length_seconds: int


class Album(BaseModel):
    """Data model for an album."""

    name: str
    artist: str
    songs: List[Song]

In [ ]:
from llama_index.output_parsers import PydanticOutputParser

prompt_template_str = """\
Generate an example album, with an artist and a list of songs. \
Using the movie {movie_name} as inspiration.\
"""
program = LLMTextCompletionProgram.from_defaults(
    output_parser=PydanticOutputParser(Album),
    prompt_template_str=prompt_template_str,
    llm=llm,
    verbose=True,
)

In [ ]:
output = program(movie_name="The Shining")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ValidationError: ignored

In [ ]:
print(output)

## Data Agent

Similar to programs, OpenAI LLMs will use `OpenAIAgent`, while other LLMs will use `ReActAgent`.

In [ ]:
from llama_index.agent import OpenAIAgent, ReActAgent

agent = ReActAgent.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)

It seems tool usage is pretty flakey

In [ ]:
response = agent.chat("Hello!")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ValueError: ignored

In [ ]:
response = agent.chat("What was mentioned about Meta? How Does it differ from how OpenAI is talked about?")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Thought: I need to use a tool to help me answer the question.
Action: summary
Action Input: {'text': 'What was mentioned about Meta? How Does it differ from how OpenAI is talked about? '}


TypeError: ignored